In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Các thư viện cần thiết
import seaborn as sns
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
%matplotlib inline

Load data từ file

In [ ]:
train_data = pd.read_csv("/kaggle/input/covid19-global-forecasting-week-5/train.csv")
test_data = pd.read_csv("/kaggle/input/covid19-global-forecasting-week-5/test.csv")

In [ ]:
# Hiển thị 10 dòng đầu trong train_data
train_data.head(10)

In [ ]:
# Hiển thị 10 dòng đầu trong test_data
test_data.head(10)

## **Đánh giá data**

Kiểm tra missing value, nhận thấy County và Province_State có nhiều dòng bị thiếu dữ liệu nên không mang lại nhiều thông tin

In [ ]:
train_data.isnull().sum()

In [ ]:
test_data.isnull().sum()

Loại bỏ các column không mang nhiều thông tin

In [ ]:
train_data=train_data.drop(columns=["County","Province_State","Id"])
test_data=test_data.drop(columns=["County","Province_State","ForecastId"])

Hiển thị lại data

In [ ]:
train_data.head(10)

In [ ]:
test_data.head(10)

## **Thống kê, mô tả data**

Số lượng quốc gia trong bảng

In [ ]:
train_data.Country_Region.nunique()

Số ngày khảo sát

In [ ]:
train_data.Date.nunique()

Top 10 nước đông dân nhất

In [ ]:
# lấy cột Country_Region và Population
df_countries_popu = train_data[["Country_Region", "Population"]]
df_countries_popu.drop_duplicates(subset=["Country_Region", "Population"], inplace=True)
df_countries_popu = df_countries_popu.groupby(["Country_Region"], as_index=False).sum()

# top 10 nước có Population cao nhất
df_top10_countries_popu = df_countries_popu.nlargest(10, "Population")
df_top10_countries_popu

Gom nhóm train_data theo Country_Region

In [ ]:
# Gom nhóm theo Country_Region
df_country_grouped=train_data.groupby(["Country_Region"], as_index=False).sum()
df_country_grouped

Top 10 nước có TargetValue lớn nhất

In [ ]:
df_top10_countries = df_country_grouped.nlargest(10, "TargetValue")
df_top10_countries[["Country_Region", "TargetValue"]]

Gom nhóm theo Country_Region (top10), Date (dùng cho visualize data)

In [ ]:
# lấy ra list tên 10 nước có tổng TargetValue lớn nhất
top10_countries = list(df_top10_countries["Country_Region"])

# Gom nhóm train_data theo top 10 nước và theo thời gian
df_top10_countries_grouped = train_data[train_data.Country_Region.isin(top10_countries)]
df_top10_countries_grouped = df_top10_countries_grouped.groupby(["Country_Region", "Date"], as_index=False).sum()
#df_top10_countries_grouped

TargetValue của top 10 nước theo ngày

In [ ]:
df_target_value_daily = pd.pivot_table(
    df_top10_countries_grouped,
    values="TargetValue",
    index=["Country_Region"],
    columns="Date"
)
df_target_value_daily

Gom nhóm theo Country_Region (top10), Date (chỉ xét Fatalities) (dùng cho visualize data)

In [ ]:
# Gom nhóm train_data (chỉ xét những dòng mà Target có giá trị Fatalities) theo top 10 nước và theo thời gian
df_fatalities = train_data[train_data.Country_Region.isin(top10_countries)]
df_fatalities = df_fatalities[df_fatalities["Target"]=="Fatalities"]
df_fatalities = df_fatalities.groupby(["Country_Region", "Date"], as_index=False).sum()
#df_fatalities

TargetValue (chỉ xét Fatalities) của top 10 nước theo ngày

In [ ]:
df_fatalities_daily = pd.pivot_table(
    df_fatalities,
    values="TargetValue",
    index=["Country_Region"],
    columns="Date"
)
df_fatalities_daily

## **Visualize Data**

Tương quan giữa các thuộc tính (column) Population, Weight, TargetValue

In [ ]:
# Xem mối tương quan giữa các thuộc tính (column) Population, Weight, TargetValue
sns.pairplot(train_data)

Biểu đồ tỉ lệ giữa tổng TargetValue (gồm ConfirmedCases và Fatalities) của các quốc gia

In [ ]:
fig = px.pie(
    df_country_grouped,
    values="TargetValue", names="Country_Region",
    title="TargetValue (gồm ConfirmedCases và Fatalities) của các nước",
    color_discrete_sequence=px.colors.sequential.Aggrnyl
)
fig.update_traces(textposition="inside", textinfo="percent+label")
fig.show()

Biểu đồ TargetValue của top 10 nước có TargetValue lớn nhất

In [ ]:
fig = px.bar(
    df_top10_countries, 
    y='TargetValue', x='Country_Region',
    text='TargetValue', color="TargetValue",
    title="Top 10 nước có TargetValue lớn nhất"
)
fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig.show()

Biểu đồ tỉ lệ giữa tổng TargetValue (chỉ xét Fatalities) của các quốc gia

In [ ]:
train_data_fatalities = train_data[train_data["Target"]=="Fatalities"]

fig = px.pie(
    train_data_fatalities,
    values="TargetValue", names="Country_Region",
    title="TargetValue (chỉ xét Fatalities) của các nước",
    color_discrete_sequence=px.colors.sequential.Aggrnyl
)
fig.update_traces(textposition="inside", textinfo="percent+label")
fig.show()

Biểu đồ tỉ lệ giữa TargetValue của ConfirmedCases và Fatalities của tất cả quốc gia

In [ ]:
fig = px.pie(
    train_data,
    values="TargetValue", names="Target",
    title="TargetValue của ConfirmedCases và Fatalities",
    color_discrete_sequence=px.colors.sequential.Emrld
)
fig.update_traces(textposition="inside", textinfo="percent+label")
fig.show()

Biểu đồ tỉ lệ Population của các nước

In [ ]:
fig = px.pie(
    df_countries_popu, 
    values="Population", names="Country_Region",
    title="Population của các nước",
    color_discrete_sequence=px.colors.sequential.Aggrnyl
)
fig.update_traces(textposition="inside", textinfo="percent+label")
fig.show()

Biểu đồ số lượng dân số của top 10 nước có Population lớn nhất

In [ ]:
fig = px.bar(
    df_top10_countries_popu,
    y='Population', x='Country_Region',
    text='Population', color="Population",
    title="Top 10 nước có Population lớn nhất")
fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig.show()

Biểu đồ thay đổi TargetValue (gồm ComfirmedCases và Fatalities) theo ngày của top10 nước có tổng TargetValue cao nhất

In [ ]:
px.line(
    df_top10_countries_grouped,
    x="Date", y="TargetValue", color="Country_Region", 
    title="TargetValue theo ngày của top 10 nước có TargetValue lớn nhất"
)

In [ ]:
fig = go.Figure(data=go.Heatmap(
        z=df_target_value_daily.values,# TargetValue
        x=df_target_value_daily.columns,# Date
        y=df_target_value_daily.index,#Country_Region
        colorscale="tempo"))

fig.update_layout(title="TargetValue theo ngày của top 10 nước có TargetValue lớn nhất")

fig.show()

Biểu đồ thay đổi TargetValue (chỉ xét Fatalities) theo ngày của top10 nước có tổng TargetValue cao nhất

In [ ]:
px.line(
    df_fatalities,
    x="Date", y="TargetValue", color="Country_Region",
    title="TargetValue (chỉ xét Fatalities) theo ngày của top 10 nước có TargetValue lớn nhất"
)

In [ ]:
fig = go.Figure(data=go.Heatmap(
        z=df_fatalities_daily.values,# TargetValue
        x=df_fatalities_daily.columns,# Date
        y=df_fatalities_daily.index,#Country_Region
        colorscale="tempo"))

fig.update_layout(title="TargetValue (chỉ xét Fatalities) theo ngày của top 10 nước có TargetValue lớn nhất")

fig.show()

## **Preprocessing data**

Load lại tập train_data và test_data

In [ ]:
train_data = pd.read_csv("/kaggle/input/covid19-global-forecasting-week-5/train.csv")
test_data = pd.read_csv("/kaggle/input/covid19-global-forecasting-week-5/test.csv")

Loại bỏ các cột mang ít thông tin

In [ ]:
train_data=train_data.drop(columns=["County","Province_State","Id"])
test_data=test_data.drop(columns=["County","Province_State","ForecastId"])

Train data trước khi chuẩn hóa

In [ ]:
train_data

Test data trước khi chuẩn hóa

In [ ]:
test_data

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

Chuẩn hóa Country_Region, Target thành số nguyên

In [ ]:
# Chuẩn hóa Country_Region thành số nguyên
train_data.iloc[:,0] = le.fit_transform(train_data.iloc[:,0].values)

# Chuẩn hóa Target thành số nguyên
train_data.iloc[:,4] = le.fit_transform(train_data.iloc[:,4].values)

Tương tự với tập test_data

In [ ]:
# Chuẩn hóa Country_Region thành số nguyên
test_data.iloc[:,0] = le.fit_transform(test_data.iloc[:,0].values)

# Chuẩn hóa Target thành số nguyên
test_data.iloc[:,4] = le.fit_transform(test_data.iloc[:,4].values)

Chuyển giá trị trong cột Date thành số nguyên theo format "Ymd", ví dụ "2020-07-10" --> 20200710

In [ ]:
dates = pd.to_datetime(train_data["Date"], format="%Y-%m-%d")
train_data["Date"] = dates.dt.strftime("%Y%m%d").astype(int)

Tương tự với tập test_data

In [ ]:
dates = pd.to_datetime(test_data["Date"], format="%Y-%m-%d")
test_data["Date"] = dates.dt.strftime("%Y%m%d").astype(int)

Train data sau khi chuẩn hóa

In [ ]:
train_data

Test data sau khi chuẩn hóa

In [ ]:
test_data

Từ train_data tạo ra tập features và tập labels

In [ ]:
# tập features: bỏ đi cột "TargetValue" từ train_data
x_train = train_data.drop("TargetValue", axis = 1)  # axis: 1 cho column, 0 cho row

# tập labels: chỉ lấy cột "TargetValue" từ train_data
y_train = train_data["TargetValue"]

In [ ]:
x_train

In [ ]:
y_train

Chia nhỏ x_train, y_train thành 2 tập con dùng để train và test

In [ ]:
from sklearn.model_selection import train_test_split 
# tập test chiếm 20%
x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=0.2, random_state=1)

## **Xây dựng mô hình**

In [ ]:
# thư viện cần thiết
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

### **Mô hình 1:** Regression sử dụng **Decision Tree**

Sử dụng pipeline để kết hợp hai bước:
* Chuẩn hóa dữ liệu đầu vào gần với phân phối có trung bình là 0 và độ lệch chuẩn là 1
* Huấn luyện dùng DecisionTreeRegressor

In [ ]:
pipe_dtr = Pipeline([
    ("scl", StandardScaler()),
    ("DecisionTreeRegressor", DecisionTreeRegressor())
])

Tiến hành huấn luyện

In [ ]:
pipe_dtr.fit(x_train, y_train)

Kiểm tra kết quả của mô hình, ta thu được độ chính xác khoảng **92%**

In [ ]:
pipe_dtr.score(x_test, y_test)

Giá trị dự đoán y_pred_dtr cho x_test

In [ ]:
y_pred_dtr = pipe_dtr.predict(x_test)

So sánh giữa y_pred_dtr và y_test

In [ ]:
fig = make_subplots(rows=2, cols=1)

fig.add_trace(go.Scatter(x=np.arange(len(y_test)), y=y_test,
                         mode='lines', name='y_test'),
             row=1, col=1)

fig.add_trace(go.Scatter(x=np.arange(len(y_test)), y=y_pred_dtr,
                         mode='lines',name='y_pred'),
             row=2, col=1)
fig.update_layout(title="So sánh y_test và y_pred")
fig.show()

### **Mô hình 2:** Regression sử dụng **Random Forest**

Sử dụng pipeline để kết hợp hai bước:
* Chuẩn hóa dữ liệu đầu vào gần với phân phối có trung bình là 0 và độ lệch chuẩn là 1
* Huấn luyện dùng RandomForestRegressor

In [ ]:
pipe_rfr = Pipeline([
    ("scl", StandardScaler()),
    ("RandomForestRegressor", RandomForestRegressor())
])

Tiến hành huấn luyện

In [ ]:
pipe_rfr.fit(x_train, y_train)

Kiểm tra kết quả của mô hình, ta thu được độ chính xác khá cao, khoảng **95%**. Kết quả này **cao hơn** Mô hình 1

In [ ]:
pipe_rfr.score(x_test, y_test)  # r2_score

Giá trị dự đoán y_pred_rfr cho x_test

In [ ]:
y_pred_rfr = pipe_rfr.predict(x_test)

So sánh giữa y_pred_rfr và y_test, ta thấy 2 đồ thị khá tương đồng nhau

In [ ]:
fig = make_subplots(rows=2, cols=1)

fig.add_trace(go.Scatter(x=np.arange(len(y_test)), y=y_test,
                         mode='lines', name='y_test'),
             row=1, col=1)

fig.add_trace(go.Scatter(x=np.arange(len(y_test)), y=y_pred_rfr,
                         mode='lines',name='y_pred'),
             row=2, col=1)
fig.update_layout(title="So sánh y_test và y_pred")
fig.show()

So sánh 2 mô hình thì ta chọn **Random Forest** vì cho độ chính xác cao hơn Decision Tree

**Nhận xét:** Random Forest có ưu điểm là nó xây dựng một tập rất nhiều decision tree. Mỗi decision tree sẽ được train trên một tập con được chọn ngẫu nhiên từ tập training ban đầu. Random Forest sẽ dựa trên kết quả của các decision tree để đưa ra kết quả tốt nhất cuối cùng. Do vậy nên thuật toán Random Forest hiệu quả hơn thuật toán Decision Tree chỉ có 1 cây.

Dự đoán trên tập test_data

In [ ]:
pred = pipe_rfr.predict(test_data)

Lưu kết quả dự đoán vào dataframe result

In [ ]:
result = pd.DataFrame({
    "Id": np.arange(pred.shape[0]),
    "TargetValue": pred
})
result

Tính kết quả cho từng quantile

In [ ]:
df_sub = result.groupby(["Id"])["TargetValue"].quantile([0.05, 0.5, 0.95]).reset_index()
df_sub.head(15)

Xử lí df_sub theo format của file submission.csv

In [ ]:
df_sub["Id"] = df_sub["Id"] + 1
df_sub["Id"] = df_sub["Id"].astype("str")
df_sub["level_1"] = df_sub["level_1"].astype("str")
df_sub["Id"] = df_sub["Id"] + "_" + df_sub["level_1"]

df_sub = df_sub.drop("level_1", axis=1)

df_sub.columns = ["ForecastId_Quantile", "TargetValue"]

#df_sub.head(15)
df_sub

Lưu kết quả vào file submission.csv (không lấy cột index)

In [ ]:
df_sub.to_csv("submission.csv", index=False)